In [101]:
import boto3
import pickle as pkl
import pandas as pd
import csv
import time
bucket = 'youtube-overlaps'

In [102]:
def see_all_files_s3(bucket):
    s3 = boto3.resource('s3')
    s3_bucket = s3.Bucket(bucket)
    files = []
    for obj in s3_bucket.objects.all():
        files.append(obj.key)
    return files

def load_pkl_obj_s3(file_name, bucket):
    try:
        s3 = boto3.resource('s3')
        obj = pkl.loads(s3.Bucket(bucket).Object(file_name).get()['Body'].read())
    except:
        time.sleep(5)
        return load_pkl_obj_s3(file_name, bucket)
        
    return obj

In [103]:
#commenter_files = see_all_files_s3(bucket)
commenter_files = ["Stanz_Full_Overlaps"]
channel_file_dict = {}
for file_name in commenter_files:
    prefix_len = 9
    suffix_len = 17
    channel_name = file_name[prefix_len:-suffix_len]
    channel_file_dict[channel_name] = file_name

len(channel_file_dict)

1

In [104]:
master_overlap_dict = {}
count = 0
for primary_channel, _file in channel_file_dict.items():
    channel_overlap_dict = load_pkl_obj_s3(_file, bucket)
    overlaps = channel_overlap_dict[list(channel_overlap_dict.keys())[0]]
    for comparison_channel in list(overlaps.keys()):
        if(overlaps[comparison_channel] < 10):
            del overlaps[comparison_channel]
    
    channel_overlap_dict[list(channel_overlap_dict.keys())[0]] = overlaps
    master_overlap_dict.update(channel_overlap_dict)

    if(count % 100 == 0):
        print(count / len(channel_file_dict))

    count += 1

print(len(master_overlap_dict))

0.0
1


In [105]:
master_overlap_dict

{'Stanz': {'--FL6OwLFWGIZfLfazY4yA': 13,
  '--LAVm36WiCmqQbekV17qQ': 164,
  '-2PGzzS1Ggq_cvZan_hqpA': 14,
  '-4KnPMmZzwAzW7SbVATUZQ': 42,
  '-4TwmWQShd0R6YZ_HJ0NfA': 31,
  '-5Yj4HIiruwkzFuLflxL_w': 12,
  '-68yVCdXC2ZJSypRV4FmVQ': 17,
  '-69vhXlCa3XHbF8JHCQHfg': 59,
  '-6N7qlNd4KIzfSy1mrR2kw': 11,
  '-ASDT9n-sUPwA7uUPN0OoQ': 47,
  '-BZEmUUUIXmD1CgkWACVyQ': 35,
  '-Bc-4nV2-Tk4Af2rqrmrjA': 39,
  '-Diq5SjWZDol8C0_E_1Y7A': 23,
  '-FVEscwDE5C0LRQVE9b32A': 17,
  '-Hm8vtPdB9DDVFEOJMwBzg': 18,
  '-JrEOZQwdI5nvlo-x_Oi0A': 31,
  '-KrJWxMSMVBbmmpNLhSBdw': 35,
  '-Kw2wQSm9O5klQtzBuDE9w': 32,
  '-MSHwNBgzO9VC-Y9MY0aNg': 21,
  '-O9o5coq8iFSVthivq__tw': 81,
  '-Seahl1De4BnRjSrNyQezw': 16,
  '-Tvz0FJd6xRvVDw_4dc_GQ': 40,
  '-UZpjx7fUGroVLfAIAKUYg': 37,
  '-ZHt5Zgadfx-B1CM63Lqew': 55,
  '-awjwg4N6lWu1TSK1ostjQ': 27,
  '-bOPhs_XA5bETT3G37WF8A': 14,
  '-bqx0CEnWLZ-o64hFvsnsQ': 16,
  '-brZ2iokMW4d208sEPcmTw': 33,
  '-chqi3Gpb4F7yBqedlnq5g': 29,
  '-eqG9dW9ydhDGs_t67URMg': 107,
  '-i2ywiuvjvpTy2zW-tXfkw': 5

In [106]:
channel_df = pd.read_csv('Top20kYoutubeChannels.csv')

In [107]:
unscraped_channels = []
def create_edges_csv(overlap_dict):
    with open('ludwig_edges.csv', 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Source', 'Target', 'Weight'])
        for primary_channel, primary_overlap_dict in master_overlap_dict.items():
            for comparison_channel, shared_overlap in primary_overlap_dict.items():
                if(comparison_channel not in overlap_dict):
                    unscraped_channels.append(comparison_channel)
                writer.writerow([primary_channel, comparison_channel, shared_overlap])

def create_labels_csv(overlap_dict, channel_df):
    with open('ludwig_labels.csv', 'w', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Id', 'Label', 'Count'])
        for primary_channel in overlap_dict:
            display_name = channel_df[channel_df['username'] == primary_channel].iloc[0]['displayname']
            subscriber_count = channel_df[channel_df['username'] == primary_channel].iloc[0]['subscribers']
            writer.writerow([primary_channel, display_name, subscriber_count])
        for primary_channel in unscraped_channels:
            display_name = channel_df[channel_df['username'] == primary_channel].iloc[0]['displayname']
            subscriber_count = channel_df[channel_df['username'] == primary_channel].iloc[0]['subscribers']
            writer.writerow([primary_channel, display_name, subscriber_count])

#This is only used if a human readable edge list is desired for ad hoc channel analysis
def create_display_name_edges_csv(master_overlap_dict, channel_df):
    with open('stanz_readable_edges.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Source', 'Target', 'Weight'])
        for primary_channel, primary_overlap_dict in master_overlap_dict.items():
            for comparison_channel, shared_overlap in primary_overlap_dict.items():
                try:
                    display_name = channel_df[channel_df['username'] == comparison_channel].iloc[0]['displayname']
                except IndexError as e:
                    display_name = comparison_channel
                writer.writerow([primary_channel, f"{display_name}", shared_overlap])

In [108]:
# create_edges_csv(master_overlap_dict)
# create_labels_csv(master_overlap_dict, channel_df)
create_display_name_edges_csv(master_overlap_dict, channel_df)